In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()
import gc
import datetime

In [2]:
basedir = './Training/Training5_20220204/cat_col_maps/'

In [3]:
filenames = [f for f in os.listdir(basedir)]
filenames

['구.xlsx',
 '대장구분명.xlsx',
 '동.xlsx',
 '부동산유형.xlsx',
 '브랜드.xlsx',
 '일괄계약.xlsx',
 '지목.xlsx']

In [4]:
cat_cols = [col.replace('.xlsx', '') for col in filenames]
cat_cols

['구', '대장구분명', '동', '부동산유형', '브랜드', '일괄계약', '지목']

In [5]:
coor_df = pd.read_csv('./Prepped Data/연속지적도출처_지번별좌표_본번부번정리_20211222.csv')
print(coor_df.shape)
coor_df.head()

(922318, 12)


,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표,글자있음,글자,지번만,본번,부번
0,강남구,개포동,0-1 가,0-1,가,206331.014484,442028.661953,0,글자없음,0-1,0,1
1,강남구,개포동,0-2 가,0-2,가,206254.177141,441986.263641,0,글자없음,0-2,0,2
2,강남구,개포동,0-3 가,0-3,가,206174.409344,441932.356344,0,글자없음,0-3,0,3
3,강남구,개포동,0-4 가,0-4,가,206631.548237,442577.897175,0,글자없음,0-4,0,4
4,강남구,개포동,0-5 가,0-5,가,205243.406000,442547.328000,0,글자없음,0-5,0,5


In [6]:
test_gu = '은평구'
test_dong = '불광동'
test_bonbun = 480
test_bubun = 230

In [7]:
test_coor_df = coor_df[
    (coor_df['시군구명'] == test_gu)
    &(coor_df['법정동명'] == test_dong)
    &(coor_df['본번'] == test_bonbun)
    &(coor_df['부번'] == test_bubun)
]
print(test_coor_df.shape)
test_coor_df.head()

(1, 12)


,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표,글자있음,글자,지번만,본번,부번
768312,은평구,불광동,480-230대,480-230,대,193102.099881,457961.426861,0,글자없음,480-230,480,230


In [8]:
training_df = pd.read_csv('./Prepped Data/training_data_20220204_2.csv')
print(training_df.shape)
training_df.head()

(2091509, 18)


,주소_건축년도,건축년도,전용면적(㎡),층,x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,부동산유형,브랜드,대장구분명,지목,구,동,일괄계약,거래/보증금(만원)
0,서울특별시 강남구 개포동 12 1992,1992,33.18,1,2.066072,4.435861,2,14,0.034422,0.999407,0,0,0,0,0,0,0,15900
1,서울특별시 강남구 개포동 12 1992,1992,39.53,5,2.066072,4.435861,6,14,0.103102,0.994671,0,0,0,0,0,0,0,24800
2,서울특별시 강남구 개포동 12 1992,1992,39.53,12,2.066072,4.435861,8,14,0.137279,0.990532,0,0,0,0,0,0,0,24700
3,서울특별시 강남구 개포동 12 1992,1992,49.86,11,2.066072,4.435861,8,14,0.137279,0.990532,0,0,0,0,0,0,0,33300
4,서울특별시 강남구 개포동 12 1992,1992,39.53,4,2.066072,4.435861,9,14,0.154309,0.988023,0,0,0,0,0,0,0,24700


In [9]:
test_df = training_df.head(0)
print(test_df.shape)
test_df.head()

(0, 18)


,주소_건축년도,건축년도,전용면적(㎡),층,x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,부동산유형,브랜드,대장구분명,지목,구,동,일괄계약,거래/보증금(만원)


In [10]:
test_df.loc[0, '주소_건축년도'] = '서울특별시 ' + test_coor_df['시군구명'].iloc[0] + ' ' + test_coor_df['법정동명'].iloc[0] + ' ' +\
str(test_coor_df['본번'].iloc[0]) + '-' + str(test_coor_df['부번'].iloc[0]) + ' 2021'

In [11]:
(pd.to_datetime(datetime.datetime.now()) - pd.to_datetime('2006-01-01')).days

5881

In [12]:
def year_circular_sine_func(x):
    return np.sin(2*np.pi*((x)/365))

def year_circular_cosine_func(x):
    return np.cos(2*np.pi*((x)/365))

In [13]:
filenames

['구.xlsx',
 '대장구분명.xlsx',
 '동.xlsx',
 '부동산유형.xlsx',
 '브랜드.xlsx',
 '일괄계약.xlsx',
 '지목.xlsx']

In [14]:
for file in filenames:
    print(file)
    cat_df = pd.read_excel(basedir + file)
    display(cat_df)

구.xlsx


,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,도봉구,동대문구,동작구,마포구,서대문구,서초구,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24


대장구분명.xlsx


,일반,산
0,0,1


동.xlsx


...

,기타,현대,삼성,한신,래미안,아이파크,편한세상,아펠바움,아크로,롯데캐슬,더샵,코오롱,SK,힐스테이트,푸르지오,레미안,LG,데시앙,써밋,디에이치,리슈,대방,호반,포레나,중흥
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24


일괄계약.xlsx


,N,Y
0,0,1


지목.xlsx


,대,도,전,공,임,잡,답,차,장,주,종,수,사,창
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13


In [15]:
training_df['계약날짜일수_2006년기준'].describe()

count    2.091509e+06
mean     2.978506e+03
std      1.726680e+03
min      0.000000e+00
25%      1.334000e+03
50%      3.307000e+03
75%      4.394000e+03
max      5.807000e+03
Name: 계약날짜일수_2006년기준, dtype: float64

In [16]:
test_df.loc[0, '건축년도'] = 2021
test_df.loc[0, '전용면적(㎡)'] = 30
test_df.loc[0, 'x좌표'] = test_coor_df['x좌표'].iloc[0] / 100000
test_df.loc[0, 'y좌표'] = test_coor_df['y좌표'].iloc[0] / 100000
test_df.loc[0, '계약날짜일수_2006년기준'] = 5807
test_df.loc[0, '계약날짜기준_건물연식'] = 0
test_df.loc[0, '계약날짜일수_sin'] = year_circular_sine_func(5807)
test_df.loc[0, '계약날짜일수_cos'] = year_circular_cosine_func(5807)
test_df.loc[0, '층'] = 1
test_df.loc[0, '부동산유형'] = '연립다세대'
test_df.loc[0, '브랜드'] = '기타'
test_df.loc[0, '대장구분명'] = '일반'
test_df.loc[0, '지목'] = '대'
test_df.loc[0, '구'] = '은평구'
test_df.loc[0, '동'] = '불광동'
test_df.loc[0, '일괄계약'] = 'Y'

In [17]:
test_df = test_df.drop(columns=['거래/보증금(만원)'])

In [18]:
test_df.head()

,주소_건축년도,건축년도,전용면적(㎡),층,x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,부동산유형,브랜드,대장구분명,지목,구,동,일괄계약
0,서울특별시 은평구 불광동 480-230 2021,2021.0,30.0,1.0,1.931021,4.579614,5807.0,0.0,-0.538005,0.842942,연립다세대,기타,일반,대,은평구,불광동,Y


In [19]:
filenames

['구.xlsx',
 '대장구분명.xlsx',
 '동.xlsx',
 '부동산유형.xlsx',
 '브랜드.xlsx',
 '일괄계약.xlsx',
 '지목.xlsx']

In [20]:
test_mapped_df = test_df.copy()

In [21]:
for file in filenames:
    map_df = pd.read_excel(basedir + file)
    file = file.replace('.xlsx', '')
    print(file)
    
    map_dict = {}
    for i in range(map_df.shape[1]):
        map_dict[map_df.columns[i]] = map_df.iloc[0, i]
    
    test_mapped_df[file] = test_mapped_df[file].map(map_dict)

구
대장구분명
동
부동산유형
브랜드
일괄계약
지목


In [22]:
test_mapped_df

,주소_건축년도,건축년도,전용면적(㎡),층,x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,부동산유형,브랜드,대장구분명,지목,구,동,일괄계약
0,서울특별시 은평구 불광동 480-230 2021,2021.0,30.0,1.0,1.931021,4.579614,5807.0,0.0,-0.538005,0.842942,2,0,0,0,21,282,1


In [23]:
os.listdir('./Training/Training5_20220204/')

['catboost_quantile0125_0',
 'catboost_quantile0125_1',
 'catboost_quantile0125_2',
 'catboost_quantile0125_3',
 'catboost_quantile0125_4',
 'catboost_quantile025_0',
 'catboost_quantile025_1',
 'catboost_quantile025_2',
 'catboost_quantile025_3',
 'catboost_quantile025_4',
 'catboost_quantile05_0',
 'catboost_quantile05_1',
 'catboost_quantile05_2',
 'catboost_quantile05_3',
 'catboost_quantile05_4',
 'catboost_quantile075_0',
 'catboost_quantile075_1',
 'catboost_quantile075_2',
 'catboost_quantile075_3',
 'catboost_quantile075_4',
 'catboost_quantile0875_0',
 'catboost_quantile0875_1',
 'catboost_quantile0875_2',
 'catboost_quantile0875_3',
 'catboost_quantile0875_4',
 'catboost_RMSE_0',
 'catboost_RMSE_1',
 'catboost_RMSE_2',
 'catboost_RMSE_3',
 'catboost_RMSE_4',
 'cat_col_maps']

In [24]:
models = [f for f in os.listdir('./Training/Training5_20220204/') if f.startswith('catboost')]
models

['catboost_quantile0125_0',
 'catboost_quantile0125_1',
 'catboost_quantile0125_2',
 'catboost_quantile0125_3',
 'catboost_quantile0125_4',
 'catboost_quantile025_0',
 'catboost_quantile025_1',
 'catboost_quantile025_2',
 'catboost_quantile025_3',
 'catboost_quantile025_4',
 'catboost_quantile05_0',
 'catboost_quantile05_1',
 'catboost_quantile05_2',
 'catboost_quantile05_3',
 'catboost_quantile05_4',
 'catboost_quantile075_0',
 'catboost_quantile075_1',
 'catboost_quantile075_2',
 'catboost_quantile075_3',
 'catboost_quantile075_4',
 'catboost_quantile0875_0',
 'catboost_quantile0875_1',
 'catboost_quantile0875_2',
 'catboost_quantile0875_3',
 'catboost_quantile0875_4',
 'catboost_RMSE_0',
 'catboost_RMSE_1',
 'catboost_RMSE_2',
 'catboost_RMSE_3',
 'catboost_RMSE_4']

In [25]:
models[-1]

'catboost_RMSE_4'

In [26]:
import catboost

In [27]:
from catboost import CatBoostRegressor

In [28]:
basedir2 = basedir.replace('cat_col_maps/', '')
basedir2

'./Training/Training5_20220204/'

In [29]:
x_test_mapped_df = test_mapped_df.drop(columns=['주소_건축년도'])
print(x_test_mapped_df.shape)
x_test_mapped_df.head()

(1, 16)


,건축년도,전용면적(㎡),층,x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,부동산유형,브랜드,대장구분명,지목,구,동,일괄계약
0,2021.0,30.0,1.0,1.931021,4.579614,5807.0,0.0,-0.538005,0.842942,2,0,0,0,21,282,1


In [30]:
training_df.head()

,주소_건축년도,건축년도,전용면적(㎡),층,x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,부동산유형,브랜드,대장구분명,지목,구,동,일괄계약,거래/보증금(만원)
0,서울특별시 강남구 개포동 12 1992,1992,33.18,1,2.066072,4.435861,2,14,0.034422,0.999407,0,0,0,0,0,0,0,15900
1,서울특별시 강남구 개포동 12 1992,1992,39.53,5,2.066072,4.435861,6,14,0.103102,0.994671,0,0,0,0,0,0,0,24800
2,서울특별시 강남구 개포동 12 1992,1992,39.53,12,2.066072,4.435861,8,14,0.137279,0.990532,0,0,0,0,0,0,0,24700
3,서울특별시 강남구 개포동 12 1992,1992,49.86,11,2.066072,4.435861,8,14,0.137279,0.990532,0,0,0,0,0,0,0,33300
4,서울특별시 강남구 개포동 12 1992,1992,39.53,4,2.066072,4.435861,9,14,0.154309,0.988023,0,0,0,0,0,0,0,24700


In [31]:
cat_cols = [col.replace('.xlsx', '') for col in filenames]
cat_cols

['구', '대장구분명', '동', '부동산유형', '브랜드', '일괄계약', '지목']

In [32]:
test_pool = catboost.Pool(x_test_mapped_df, cat_features=cat_cols)

In [33]:
models

['catboost_quantile0125_0',
 'catboost_quantile0125_1',
 'catboost_quantile0125_2',
 'catboost_quantile0125_3',
 'catboost_quantile0125_4',
 'catboost_quantile025_0',
 'catboost_quantile025_1',
 'catboost_quantile025_2',
 'catboost_quantile025_3',
 'catboost_quantile025_4',
 'catboost_quantile05_0',
 'catboost_quantile05_1',
 'catboost_quantile05_2',
 'catboost_quantile05_3',
 'catboost_quantile05_4',
 'catboost_quantile075_0',
 'catboost_quantile075_1',
 'catboost_quantile075_2',
 'catboost_quantile075_3',
 'catboost_quantile075_4',
 'catboost_quantile0875_0',
 'catboost_quantile0875_1',
 'catboost_quantile0875_2',
 'catboost_quantile0875_3',
 'catboost_quantile0875_4',
 'catboost_RMSE_0',
 'catboost_RMSE_1',
 'catboost_RMSE_2',
 'catboost_RMSE_3',
 'catboost_RMSE_4']

In [34]:
rmse_models_list = [model for model in models if 'RMSE' in model]
rmse_models_list

['catboost_RMSE_0',
 'catboost_RMSE_1',
 'catboost_RMSE_2',
 'catboost_RMSE_3',
 'catboost_RMSE_4']

In [35]:
models_list = []
for name in rmse_models_list:
    model = CatBoostRegressor().load_model(basedir2 + name)
    models_list.append(model)

In [36]:
preds_list = []
for model in models_list:
    preds = model.predict(test_pool)
    print(preds)
    preds_list.append(preds)

[27620.62084377]
[27207.85154974]
[26303.36040324]
[26449.3922927]
[27621.58627954]


In [37]:
np.mean(preds_list)

27040.56227379825

In [38]:
np.mean(preds_list) /  30

901.3520757932749

In [39]:
quantile05_model_names_list = [model for model in models if 'quantile05' in model]
quantile05_model_names_list

['catboost_quantile05_0',
 'catboost_quantile05_1',
 'catboost_quantile05_2',
 'catboost_quantile05_3',
 'catboost_quantile05_4']

In [40]:
quantile05_models_list = []
for name in quantile05_model_names_list:
    model = CatBoostRegressor().load_model(basedir2 + name)
    quantile05_models_list.append(model)

In [41]:
quantile05_models_list

In [42]:
preds_list = []
for model in quantile05_models_list:
    preds = model.predict(test_pool)
    preds_list.append(preds)

In [43]:
preds_list

[array([25439.0449392]),
 array([26377.80679935]),
 array([24152.35807225]),
 array([24492.28614373]),
 array([24157.70464776])]

In [44]:
np.mean(preds_list)

24923.840120455556

In [45]:
np.mean(preds_list) / 30

830.7946706818518

In [ ]:
quantile075_model_names_list = [model for model in models if 'quantile075' in model]
quantile075_model_names_list

In [ ]:
quantile075_models_list = []
for name in quantile075_model_names_list:
    model = CatBoostRegressor().load_model(basedir2 + name)
    quantile075_models_list.append(model)

In [ ]:
quantile075_models_list

In [ ]:
preds_list = []
for model in quantile075_models_list:
    preds = model.predict(test_pool)
    preds_list.append(preds)

In [ ]:
preds_list

In [ ]:
np.mean(preds_list)

In [ ]:
np.mean(preds_list) / 20

In [ ]:
quantile025_model_names_list = [model for model in models if 'quantile025' in model]
quantile025_model_names_list

In [ ]:
quantile025_models_list = []
for name in quantile025_model_names_list:
    model = CatBoostRegressor().load_model(basedir2 + name)
    quantile025_models_list.append(model)

In [ ]:
quantile075_models_list

In [ ]:
preds_list = []
for model in quantile025_models_list:
    preds = model.predict(test_pool)
    preds_list.append(preds)

In [ ]:
preds_list

In [ ]:
np.mean(preds_list)

In [ ]:
np.mean(preds_list) / 20